In [1]:
import os
from src.core.config import get_settings
os.chdir('/Users/thomas/PycharmProjects/FDE/classifier')
get_settings.cache_clear()
settings = get_settings()

In [2]:
from src.utils.tools import retrieve

In [3]:
retrieve('sefe')


[Document(id='f11b169e-876e-42d6-8a7a-99097b7c791c', metadata={'digest': 'vI7EHYpQg6bnz2PsLviZVeneXbMs9iqDQyOgUjIhClc=', 'parser': 'DoclingParser', 'source': 'sefe.pdf', 'dl_meta': {'origin': {'filename': 'tmp6m9xs_kr.pdf', 'mimetype': 'application/pdf', 'binary_hash': 4837886699614833239}, 'version': '1.0.0', 'headings': ['Risikomanagementsystem'], 'doc_items': [{'prov': [{'bbox': {'b': 231.03999999999996, 'l': 66.0, 'r': 547.784, 't': 295.344, 'coord_origin': 'BOTTOMLEFT'}, 'page_no': 10, 'charspan': [0, 459]}], 'label': 'text', 'parent': {'$ref': '#/body'}, 'children': [], 'self_ref': '#/texts/103', 'content_layer': 'body'}, {'prov': [{'bbox': {'b': 170.44000000000005, 'l': 66.0, 'r': 547.747, 't': 220.82400000000007, 'coord_origin': 'BOTTOMLEFT'}, 'page_no': 10, 'charspan': [0, 355]}], 'label': 'text', 'parent': {'$ref': '#/body'}, 'children': [], 'self_ref': '#/texts/104', 'content_layer': 'body'}, {'prov': [{'bbox': {'b': 82.12, 'l': 66.0, 'r': 547.72, 't': 160.10400000000004, 'c